## Performing sentiment analysis using DistilBERT

Information about the fine-tuned version of DistilBERT can be found [here](https://huggingface.co/noahnsimbe/DistilBERT-yelp-sentiment-analysis)


In [1]:
from transformers import pipeline

## Loading the model

In [2]:
model = pipeline(model="noahnsimbe/DistilBERT-yelp-sentiment-analysis")

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

## Performing sentiment analysis

In [3]:
model(["Been going to Dr. Goldberg for over 10 years. I think I was one of his 1st patients when he started at MHMG. He's been great over the years and is really all about the big picture. It is because of him, not my now former gyn Dr. Markoff, that I found out I have fibroids. He explores all options with you and is very patient and understanding. He doesn't judge and asks all the right questions. Very thorough and wants to be kept in the loop on every aspect of your medical health and your life."])

[{'label': 'LABEL_2', 'score': 0.992021381855011}]

`LABEL_0`, `LABEL_1`, `LABEL_2` stand for `Negative`, `Neutral` and `Positive` respectively